## Q1. Running Elastic

In [9]:
!curl 'localhost:9200'

{
  "name" : "c40ee03c6697",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "wJbLyffZR565mSBH36S4NA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


#### Imports

In [164]:
import os
import requests 
from tqdm import tqdm
from elasticsearch import Elasticsearch

## Q2. Getting the data

In [25]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [27]:
es_client = Elasticsearch('http://localhost:9200')

In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [31]:
index_name = "course-questions"

In [36]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
def index_documents(documents):
    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)

In [38]:
index_documents(documents)

100%|███████████████████████████████████████████| 948/948 [00:22<00:00, 42.93it/s]


## Q3. Searching

In [70]:
def elastic_search(query, size=3, filters=[]):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": [{"term": word} for word in filters]
            }
        }
    }

    search_results = es_client.search(index=index_name, body=search_query)
    return search_results['hits']['hits']

In [147]:
query = "How do I execute a command in a running docker container?"

In [148]:
top_score = elastic_search(query)[0]['_score']
print(f"The score for the top ranking result is {round(top_score, 2)}")

The score for the top ranking result is 84.05


## Q4. Filtering
Limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

In [149]:
new_es_query = elastic_search(query, 3, [{"course": "machine-learning-zoomcamp"}])

In [150]:
print(new_es_query[2]["_source"]["question"])

How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

In [151]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [152]:
context = ""

In [153]:
for doc in new_es_query:
  context += context_template.format(question=doc["_source"]["question"], text=doc["_source"]["text"]) + "\n\n"

In [154]:
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [155]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [156]:
prompt = prompt_template.format(question=query, context=context).strip()

In [157]:
len(prompt)

1462

## Q6. Tokens

In [162]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [163]:
len(encoding.encode(prompt))

322